<a href="https://colab.research.google.com/github/jeffreyboschman/heartAttackStats/blob/main/SQL%2BmachineLearningExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up database (SQLite)

This was my first time using SQL, so I wanted to practice and understand what is going on. I decided to use SQLite from the MacOS terminal command line.
1. The first step is to make sure SQLite installed. You can check the version with `sqlite3 --version` 
  - It should come bundled with Python, but if not you can do `brew install sqlite3`
2. The data that I want to put into a database is a heart attack analysis dataset from Kaggle (https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset). I downloaded them as csv files.
3. Next we want to convert the .csv files into databases.

  - 3a. Make the SQLite database
    - `$ sqlite3 stats.db` -> from the command line, creates a new database called `stats.db` and opens SQLite. In SQLite, all new commands will have the prompt `sqlite>` (see https://sqlite.org/cli.html)
    
    - Some useful SQLite dot-commands (not general SQL, only for SQLite): https://www.tutorialspoint.com/sqlite/sqlite_commands.htm
      - `.databases` -> to see what databases you have created
      - `.tables` -> to see what tables you have created
      - `.show` -> show the current values of the various settings
      - `.header on` -> turns header setting on
      - `.mode column` -> sets output mode to column
      - `.timer on` -> to see how long operations takes
      - `.quit` or `.exit` -> close SQLite

  - 3b. Create a new, empty table with the correct column names and datatypes
  
    - ```sqlite3
    sqlite> CREATE TABLE heart(
    ...> age INTEGER,
    ...> sex INTEGER,
    ...> cp INTEGER,
    ...> trtbps INTEGER,
    ...> chol INTEGER,
    ...> fbs INTEGER,
    ...> restecg INTEGER,
    ...> thalachh INTEGER,
    ...> exng INTEGER,
    ...> oldpeak FLOAT,
    ...> slp INTEGER,
    ...> caa INTEGER,
    ...> thall INTEGER,
    ...> output INTEGER
    ...> );
    ``` 

  - 3c. Import the csv file `heart.csv` into the existing table called `heart`
    - `.mode csv` -> change to csv mode
    - `.import heart.csv heart`
    - `.schema heart` -> verify the structure of the table `heart` by showing the matching CREATE statement
    - `SELECT * FROM heart;` -> verify the table contains the data from the csv. But oh wait, you may notice that the first line of the csv is imported as well as an entry.
    - `DELETE FROM heart LIMIT 1;` -> delete the first row
    - `SELECT * FROM heart;` -> now you can see that the first row is just values
  
  - 3b. Close SQLite
    - `.quit` or `.exit`

4. Finally, we want to make sure we can access the `stats.db` database file later using this Google Colab / Jupyter Notebook. I'm just going to add it to a git respository. 


# Accessing database (SQLite in Python) and making Pandas dataframe

Import some libraries (`sqlite3` is part of the standard library) and get the `stats.db` file from the last the step, which I put on git.

In [1]:
import sqlite3
import pandas as pd

!rm -rf heartAttackStats
!git clone https://github.com/jeffreyboschman/heartAttackStats.git

Cloning into 'heartAttackStats'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 43 (delta 19), reused 17 (delta 3), pack-reused 0
Unpacking objects: 100% (43/43), done.


Establish a connection to the stats.db file, and (optional) confirm the table is inside it.

In [18]:
conn = sqlite3.connect('heartAttackStats/data/stats.db')
print("Opened database successfully");

cur = conn.cursor()  #creates a "cursor" that we can use to execuse SQL commands
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
conn.close() #it is good practice to close the connection (even though my next step will need it, it is in a different code block)

Opened database successfully
[('heart',)]


Directly create a Pandas dataframe from the SQL `stats.db` table named `heart`

In [22]:
conn = sqlite3.connect('heartAttackStats/data/stats.db')
heart_df = pd.read_sql_query("SELECT * FROM heart", conn)
conn.close()

heart_df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
1,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
2,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
3,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
4,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
300,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
301,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
302,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


## Other method: Creating Pandas dataframe from csv directly

Ya I also could have just created the Pandas dataframe directly from the csv file, but like I said, I wanted to practice SQL and I already know Pandas pretty well.

In [23]:
heart_csv = pd.read_csv('/content/heartAttackStats/data/heart.csv')   
heart_df2 = pd.DataFrame(heart_csv)

heart_df2

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


# Next steps